# Tools and Routing

LangChain has its own tools ( math, search, sql etc). Also, one can create their own tools. The main idea is to write function and choose ehich function to call.

What is tools and why is it used? 

In the following we are going to explore why tools are used. 

1. The tools provice a schema to be used in the functionality and services of LLM that can be utilized to get extended capabilities.

In [4]:


import os  
import openai 
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("raj.env"))
openai.api_key=os.getenv("OPENAI_API_KEY")

In [5]:
#!pip install langchain
from langchain.agents import tool

# Define a Simple tool

In [6]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [7]:
search.name

'search'

In [8]:
search.description

'search(query: str) -> str - Search for weather online'

In [9]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

# Use Pydantic to create the function

In [10]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")
        

In [11]:
## Notice the args_schema
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [12]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [13]:
search.run("sf")

'42f'

# A weather report API

    
 

In [15]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")


'''
Notice that in the following we are using the api of an weather website where the input is longitude and latitude which are
designed as an args to the base class above and later used in the scema for tools.
With tools schema we have name,description and args of the function. We can later call the 'format_tool_to_openai_function'
to convert it to functions that can be passed as a paratmeter to the LLM call.
'''        
        
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [16]:
get_current_temperature.name

'get_current_temperature'

In [17]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [18]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [19]:
from langchain.tools.render import format_tool_to_openai_function

In [20]:
## Convert it to a list of Open AI functions calls

format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [21]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 21.9°C'

In [ ]:
#!pip install wikipedia

# Wikipedia API

In [22]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [23]:
search_wikipedia.name

'search_wikipedia'

In [24]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [25]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [26]:
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: OpenAI\nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the artificial intelligence spring.The organization consists of the n

In [27]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [28]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [29]:
spec = OpenAPISpec.from_text(text)

In [30]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [31]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [32]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [33]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{\n  "params": {\n    "limit": 3\n  }\n}'}})

In [34]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{\n  "path_params": {\n    "petId": "42"\n  }\n}'}})

In [35]:
pet_callables

<function langchain.chains.openai_functions.openapi.openapi_spec_to_openai_fn.<locals>.default_call_api(name: 'str', fn_args: 'dict', headers: 'Optional[dict]' = None, params: 'Optional[dict]' = None, **kwargs: 'Any') -> 'Any'>

Notice that in the aboove 'text' there are 3 different paths 'listPets', 'createPets' and 'showPetById'. 

Next we call the  spec = OpenAPISpec.from_text(text) and pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec). And we get 3 seperate functions. Later we invoke the openai chat function with different questions and notice the parameters.  
This is the power of the functions. It recognize the functions to call and get args accordingly.

# Routing


In the above we used the tools and how to convert them to open ai function. Next we are exploring the idae of how the llm choose which path to take and whick function to call based on the input.

In the following we are using the wikipedia and the weather api and convert them to functions and asking qustions to the llm and evaluating the response.

In [36]:

functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [37]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [38]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n  "query": "langchain"\n}'}})

In [39]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [40]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

Next we are going to use an OutputParser. 

We invistigate the type , the input ,the tool and the output with the input.

In [41]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [42]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [43]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [44]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [45]:
result.tool

'get_current_temperature'

In [46]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [47]:
get_current_temperature(result.tool_input)

'The current temperature is 13.0°C'

In [48]:
result = chain.invoke({"input": "hi!"})

In [49]:
type(result)

langchain.schema.agent.AgentFinish

In [50]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

What is the difference between the AgentAction and AgentFinish?

When a function is called it is called AgentAction and a normal response is called AgentFinish.


In [51]:
'''
Here we are first checking if a fnction is being called or not. If it is being called then
we are going to use the tool choosen by the llm with the parameter as function. 
'''

from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [52]:
'''
First we design the prompt and choose the model.

OpenAIFunctionsAgentOutputParser() will return the results and we check if a function is called or not ( AgentFinish/ AgentAction)
Next we call the required function with parameter and get the output.
'''
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [53]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [54]:
result

'The current temperature is 13.0°C'

In [55]:
result = chain.invoke({"input": "What is langchain?"})

In [56]:
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: OpenAI\nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the artificial intelligence spring.The organization consists of the n

In [57]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'

# Note:


1. How to use tool and simple tools.